## Install Packages

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install facenet-pytorch

In [3]:
# requirements_file_path = "/content/drive/MyDrive/Masters/Prep/Projects/OneShot_JPMC/one-shot-face-recognition/src/requirements.txt"

# !cat "/content/drive/MyDrive/Masters/Prep/Projects/OneShot_JPMC/one-shot-face-recognition/src/requirements.txt" | xargs -n 1 pip install

In [4]:
from facenet_pytorch import MTCNN, InceptionResnetV1, training, fixed_image_standardization
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
import zipfile 
import torch
from PIL import Image
from torch.utils.data import Dataset
import torch.optim as optim
from torchvision import transforms
from sklearn.metrics import accuracy_score
import src
from tqdm.notebook import tqdm
from src.utils.celeba_helper import CelebADataset, CelebAClassifier, save_file_names, CelebADatasetTriplet
from src.utils.loss_functions import TripletLoss
from imp import reload
import shutil
from torchsummary import summary

workers = 0 if os.name == 'nt' else 2

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Filtering the label file after MTCNN face extraction (enable if you don't have the updated file)

In [8]:
# orig_mapping_file = 'data/identity_CelebA_train_test_split.txt'
# img_folder = 'data/img_align_celeba_mtcnn'

# label_df = pd.read_csv(orig_mapping_file, header=None, sep=" ", names=["file_name", "person_id", "is_train"])

# count=0
# files = []
# for filename in os.listdir(img_folder):
#     files.append(filename)

# file_names = label_df[label_df["file_name"].isin(files)]
# file_names.to_csv("data/identity_CelebA_train_test_split_mtcnn.txt", sep=" ", index=False, header=False)

# Define CelebA Dataset and Loader

In [6]:
## Load the dataset
# Path to directory with all the images
img_folder = 'data/img_align_celeba_mtcnn'
mapping_file = 'data/identity_CelebA_train_test_split_mtcnn.txt'

# Spatial size of training images, images are resized to this size.
image_size = 160
transform=transforms.Compose([
    transforms.Resize(image_size),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

# Load the dataset from file and apply transformations
celeba_dataset = CelebADatasetTriplet(img_folder, mapping_file, transform)

Image names size is: 202292


In [7]:
len(celeba_dataset.test_df)

192135

In [9]:
## Create a dataloader
# Batch size during training
batch_size = 128
# Number of workers for the dataloader
num_workers = 0 if device.type == 'cuda' else 2
# Whether to put fetched data tensors to pinned memory
pin_memory = True if device.type == 'cuda' else False

celeba_dataloader = torch.utils.data.DataLoader(celeba_dataset,  # type: ignore
                                                batch_size=batch_size,
                                                num_workers=num_workers,
                                                pin_memory=pin_memory,
                                                shuffle=False)

# FaceNet Training Pipeline

## Initializing the resnet model, optimizer and loss function

In [10]:
margin = 0.5
gamma = 0.1
lr = 0.1
epochs = 200

schedule = [40, 80, 130, 160]
str_schedule = "_".join(map(str, schedule)) #'30_50_70_80'


resnet = InceptionResnetV1(pretrained='vggface2').to(device)

## Freezing all the layers except last layer

In [11]:
# params = resnet.state_dict()
for name, param in resnet.named_parameters():
    if param.requires_grad == False:
        print(name)

In [12]:
def set_parameter_requires_grad(model):
    for name, param in model.named_parameters():
        if "last" not in name:
            param.requires_grad = False

# params = list(model.LAST_LAYER.parameters())
# optimizer = torch.optim.SGD(params, lr=lr)
set_parameter_requires_grad(resnet)

In [13]:
for name, param in resnet.named_parameters():
    if param.requires_grad == True:
        print(name)

last_linear.weight
last_bn.weight
last_bn.bias


## Initializing optimizer and loss functions

In [14]:
optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=lr)
criterion = TripletLoss(margin=margin)

# multistep LR scheduler
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=schedule, gamma=gamma)

## Test before training

In [15]:
resnet.eval().to(device)

test_anchor, test_pos, test_neg, anchor_label = celeba_dataset[1]
# test_anchor, test_pos, test_neg, anchor_label = test_anchor[1], test_pos[1], test_neg[1], anchor_label[1]
print(test_anchor.shape)

test_anchor_emb = resnet(test_anchor[None, :].to(device))
test_pos_emb = resnet(test_pos[None, :].to(device))
test_neg_emb = resnet(test_neg[None, :].to(device))
print(test_anchor[None, :].shape)

pos_dist = criterion.cal_distance(test_anchor_emb, test_pos_emb)
neg_dist = criterion.cal_distance(test_anchor_emb, test_neg_emb)

print("The distance between anchor and positive: {}".format(pos_dist[0]))
print("The distance between anchor and negative: {}".format(neg_dist[0]))

torch.Size([3, 160, 160])
torch.Size([1, 3, 160, 160])
The distance between anchor and positive: 0.6588835716247559
The distance between anchor and negative: 1.9983004331588745


## Training steps

In [ ]:
resnet.train()

loss_total = []
learning_rates = []

for epoch in tqdm(range(epochs), desc="Epochs", leave=True, position=0):
    running_loss = []
    for step, (anchors, positives, negatives, labels) in enumerate(tqdm(celeba_dataloader, 
                                                desc="Training", position=1, leave=False)):
        anchors = anchors.to(device)
        positives = positives.to(device)
        negatives = negatives.to(device)
        if anchors.shape[0] == 1:
            continue

        optimizer.zero_grad()

        anchor_emb = resnet(anchors)
        positive_emb = resnet(positives)
        negative_emb = resnet(negatives)

        loss = criterion(anchor_emb, positive_emb, negative_emb)
        # print("loss is {}".format(loss))
        loss.backward()
        optimizer.step()

        running_loss.append(loss.cpu().detach().numpy())
        # if step > 50:
        #     break
        
    loss_total.append(np.mean(running_loss))
    learning_rates.append(optimizer.param_groups[0]["lr"])
    scheduler.step()
    print("Epoch: {}/{} - Loss: {:.4f}".format(epoch, epochs, np.mean(running_loss)))


Epochs:   0%|          | 0/200 [00:00<?, ?it/s]

Training:   0%|          | 0/80 [00:00<?, ?it/s]

Epoch: 0/200 - Loss: 0.0551


Training:   0%|          | 0/80 [00:00<?, ?it/s]

Epoch: 1/200 - Loss: 0.0382


...

Training:   0%|          | 0/80 [00:00<?, ?it/s]

Epoch: 7/200 - Loss: 0.0316


Training:   0%|          | 0/80 [00:00<?, ?it/s]

Epoch: 8/200 - Loss: 0.0316


Training:   0%|          | 0/80 [00:00<?, ?it/s]

## Plotting Loss curve

In [ ]:
# printing loss function
plt.plot(loss_total)
plt.xlabel("Epochs")
plt.ylabel("TripletLoss")
plt.title("Training Triplet Loss")
plt.savefig(f"loss_curves/loss_curve_MTCNN_epoch{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.png")
plt.show()

## Plotting Learning rates with epochs

In [ ]:
# printing loss function
plt.plot(learning_rates)
plt.xlabel("Epochs")
plt.ylabel("Learning Rate")
plt.title("Learning Rate")
plt.savefig(f"loss_curves/learning_curve_MTCNN_epoch{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.png")
plt.show()

In [ ]:
model_path = f"models/facenet_model_epochs{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.pth"
if not os.path.exists(model_path):
    torch.save(resnet, model_path)

In [ ]:
model_state_path = f"models/facenet_model_statedict_epochs{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.pth"
if not os.path.exists(model_state_path):
    torch.save(resnet.state_dict(), model_state_path)

In [ ]:
# resnet = torch.load(model_path)

## Testing the trained model:

In [ ]:
resnet.eval().to(device)

test_anchor, test_pos, test_neg, anchor_label = celeba_dataset[1]
# test_anchor, test_pos, test_neg, anchor_label = test_anchor[1], test_pos[1], test_neg[1], anchor_label[1]

test_anchor_emb = resnet(test_anchor[None, :].to(device))
test_pos_emb = resnet(test_pos[None, :].to(device))
test_neg_emb = resnet(test_neg[None, :].to(device))

pos_dist = criterion.cal_distance(test_anchor_emb, test_pos_emb)
neg_dist = criterion.cal_distance(test_anchor_emb, test_neg_emb)

print("The distance between anchor and positive: {}".format(pos_dist[0]))
print("The distance between anchor and negative: {}".format(neg_dist[0]))

In [ ]:
test_anchor_emb.shape

## Accuracy of the model

In [ ]:
celeba_dataset.train_df.head()

In [ ]:
celeba_dataset.train_df.iloc[0]["file_name"]

## Creating Vault folder and vault mapping file

In [ ]:
vault_path = "data/oneshot_vault"
label_file = "data/identity_vault_person.txt"

In [ ]:
# creating the vault and test label file
if not os.path.exists(vault_path):
    os.makedirs(vault_path)

    # copying train images in the vault location and appending the label file
    with open(label_file, "w") as v_file:
        for i in range(len(celeba_dataset)):
            file = celeba_dataset.train_df.iloc[i]["file_name"]
            label = str(celeba_dataset.train_df.iloc[i]["person_id"])
            v_file.write(file+" "+label+"\n")

            # copying the file to the new folder
            src_file = os.path.join(img_folder, file)
            dst_file = os.path.join(vault_path, file)
            shutil.copy(src_file, dst_file)
        

### Creating vault embeddings or load vault embeddings if already created

In [ ]:
# function to create embeddings    
def create_embeddings(celeba_dataloader, model):
    # initializing embedding vector and gt_label list
    embeddings = torch.tensor([])
    gt_labels = []
    
    # creating embeddings 
    for step, (anchors, positives, negatives, labels) in enumerate(tqdm(celeba_dataloader, 
                                                            desc="Training", position=1)):
        anchors = anchors.to(device)
        img_embs = model(anchors).detach().cpu()
        
        embeddings = torch.cat([embeddings, img_embs])
        gt_labels.extend(labels)

    return embeddings, gt_labels

In [ ]:
vault_embeddings_file = f"pytorch_objects/vault_embeddings_epochs{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.pickle"
vault_gt_labels_file = f"pytorch_objects/vault_gt_labels_epochs{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.pickle"

# setting model to eval mode
resnet.eval().to(device)

if not os.path.exists(vault_embeddings_file) or not os.path.exists(vault_gt_labels_file):
    embeddings, gt_labels = create_embeddings(celeba_dataloader = celeba_dataloader,
                                              model = resnet)
    
    torch.save(embeddings, vault_embeddings_file)
    torch.save(gt_labels, vault_gt_labels_file)
else:
    embeddings = torch.load(vault_embeddings_file)
    gt_labels = torch.load(vault_gt_labels_file)

In [ ]:
# if not torch.is_tensor(gt_labels):
#     gt_labels = torch.tensor(gt_labels).to(device)
# else:
#     gt_labels = gt_labels.to(device)
# gt_labels.shape

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier
embeddings = embeddings.detach().cpu()
# gt_labels = gt_labels.detach().cpu()
gt_labels = torch.tensor(gt_labels)

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(embeddings, gt_labels)
# score = knn.score(test_embeddings, test_labels)

# print(f'Pre-trained model: Accuracy = {score}.')

In [ ]:
# import torch
# a = torch.rand(2,3)[None,:].transpose(2,1)#[None,:].transpose(2,1)
# b = torch.rand(4,3)[:, :, None]#.transpose(2,1)
# c=a-b
# print(f"a is: {a}")
# print(f"b is: {b}")

In [ ]:
a = torch.rand(1,512)

In [ ]:
# d = c.pow(2).sum(axis=1).transpose(1,0)
# torch.argmin(d, axis=1)

In [ ]:
# Test image:

def calculate_label(test_images, embeddings, gt_labels, embedding_model):
    # test_image_file = "s1_9.pgm"
    test_images = test_images.to(device)

    test_img_emb = embedding_model(test_images).detach().cpu()
    test_img_emb = test_img_emb[None,:].transpose(2,1).to(device)

    distance_mat = (test_img_emb - embeddings).pow(2).sum(axis=1).transpose(1,0)
    test_label_pred = gt_labels[torch.argmin(distance_mat, axis=1)]

    return test_label_pred


In [ ]:
gt_labels.device

In [ ]:
# celeba_dataset.is_train = False
# # test_gt_labels = torch.tensor([]).type(torch.int)

# test_imgs, test_labels = next(iter(celeba_dataloader))
# print(f"Shape of test_imgs: {test_imgs.shape}")
# print(f"test labels: {test_labels}")

# # test_gt_labels = torch.cat([test_gt_labels, test_labels])
# calculate_label(test_imgs, embeddings_, torch.tensor(gt_labels))

In [ ]:
torch.is_tensor(gt_labels)

In [ ]:
celeba_dataset.is_train = False

test_predictions = torch.tensor([]).type(torch.int)
test_gt_labels = torch.tensor([]).type(torch.int)

test_embeddings = torch.tensor([])

# if not torch.is_tensor(gt_labels):
#     gt_labels = torch.tensor(gt_labels).to(device)
# else:
#     gt_labels = gt_labels.to(device)
gt_labels = gt_labels.detach().cpu()

# embeddings_ = embeddings[:, :, None].to(device)

for i, (test_imgs, test_labels) in enumerate(tqdm(celeba_dataloader, 
                                desc="Training", position=1, leave=False)):
    test_gt_labels = torch.cat([test_gt_labels, test_labels])
    
    #new
    test_imgs = test_imgs.to(device)
    test_embs = resnet(test_imgs).detach().cpu()
    test_embeddings = torch.cat([test_embeddings, test_embs])
    
#     test_pred_labels = calculate_label(test_images = test_imgs, embeddings = embeddings_,
#                                        gt_labels=gt_labels, embedding_model=resnet).detach().cpu()
    
#     test_predictions = torch.cat([test_predictions, test_pred_labels])
    
    if i>200:
        break

In [ ]:
test_embeddings_file = f"pytorch_objects/test_embeddings_epochs_epochs{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.pickle"
test_gt_labels_file = f"pytorch_objects/test_gt_labels_epochs{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.pickle"

if not os.path.exists(test_embeddings_file) or not os.path.exists(test_gt_labels_file):
    torch.save(test_embeddings, test_embeddings_file)
    torch.save(test_gt_labels, test_gt_labels_file)
# test_embeddings = torch.load(test_embeddings_file)
# test_gt_labels = torch.load(test_gt_labels_file)

In [ ]:
score = knn.score(test_embeddings, test_gt_labels)

print(f'trained model: Accuracy = {score}.')
# trained model: Accuracy = 0.44198638613861385.

In [ ]:
# accuracy = test_predictions == test_gt_labels
# accuracy = accuracy.int().sum()/len(accuracy)
# print(f'Accuracy for the model: {accuracy}')



In [ ]:
test_predictions[:10]

In [ ]:
test_gt_labels[:10]